## SigNet Package test

**General idea** : test the functions defined in the module1.py file and make a first application.

**General plan** : 
1. Import the data using the DropBox dataset

2. Repeat the clustering several times to get several clustering outcomes

3. Compute the weights of each stocks within each cluster 

4. Compute the returns of each portfolio, where one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) **in this notebook, we first fix the number of clusters to be equal to 5**

Here are the main package we use in this notebook

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import requests 
from pypfopt.efficient_frontier import EfficientFrontier

In [2]:
pip -q install git+https://github.com/robertmartin8/PyPortfolioOpt.git

Note: you may need to restart the kernel to use updated packages.


We also import the module1.py file to use the function we defined there.

In [3]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
module_1_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(module_1_directory)

# Maintenant, vous pouvez importer module1
import module1

We also import the adjency.py file to use the function we defined there.

In [4]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
adjency_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(adjency_directory)

# Maintenant, vous pouvez importer module1
from adjency import Adjency

### 1. Importation of data

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for **496** assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

In [8]:
## beware of the path if Jerome or Mohamed use it 

adjency = Adjency(pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'))

In [9]:
adjency.clean_data()

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/adjency.py:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)## open
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/adjency.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float) ## close


#### Idea - Similarity

A popular similarity measure in the literature is given by the **Pearson correlation coefficient** that measures linear dependence between variables and takes values in [−1, 1]. By interpreting the correlation matrix as a weighted network whose (signed) edge weights capture the pairwise correlations, we cluster the multivariate time series by clustering the underlying signed network. 

Pearson's correlation coefficient, when applied to a [[sample (statistics)|sample]], is commonly represented by $r_{xy}$ and may be referred to as the ''sample correlation coefficient'' or the ''sample Pearson correlation coefficient''. We can obtain a formula for $r_{xy}$ by substituting estimates of the covariances and variances based on a sample into the formula above. Given paired data $\left\{ (x_1,y_1),\ldots,(x_n,y_n) \right\}$ consisting of $n$ pairs, $r_{xy}$ is defined as

\begin{align}
r_{xy} =\frac{\sum ^n _{i=1}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum ^n _{i=1}(x_i - \bar{x})^2} \sqrt{\sum ^n _{i=1}(y_i - \bar{y})^2}}
\end{align}

where
- $n$ is sample size
- $x_i, y_i$ are the individual sample points indexed with $i$
- $\bar{x} = \frac{1}{n} \sum_{i=1}^n x_i$ (the sample mean); and analogously for $\bar{y}$.

In [10]:
AA = np.array(data.iloc[0,0].replace('[', '').replace(']', '').split(', '), dtype=float)
ABM = np.array(data.iloc[1,0].replace('[', '').replace(']', '').split(', '), dtype=float)

x = np.corrcoef(AA, ABM)[0,1]

NameError: name 'data' is not defined

In [31]:
AA1 = np.array(data.iloc[0,2].replace('[', '').replace(']', '').split(', '), dtype=float)
ABM1 = np.array(data.iloc[1,2].replace('[', '').replace(']', '').split(', '), dtype=float)

y = np.corrcoef(AA1, ABM1)[0, 1]

In [54]:
## We want to create a dataframe with one column containing the returns 
## of each of the stocks

n, p = df.shape

df2 = pd.DataFrame(index=data.index, columns=['return', 'volume'])
df2['volume'] = data['volume']
for i in range(n):
    x = np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)## open
    y = np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float) ## close
    z = (y - x) / x
    df2.iloc[i, 0] = str(z)


/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_32876/148179349.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(data.iloc[i][0].replace('[', '').replace(']', '').split(', '), dtype=float)## open
/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/ipykernel_32876/148179349.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = np.array(data.iloc[i][1].replace('[', '').replace(']', '').split(', '), dtype=float) ## close


In [12]:
pearson = adjency.pearson_corr_volume()

IndexError: index 2 is out of bounds for axis 0 with size 2